In [2]:
import datajoint as dj
import numpy as np

In [5]:
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")  
ndap100 = dj.create_virtual_module("ndap100","microns_ndap100") 

In [7]:
# Source presyns and postsyns
axon_segments = (ta3p100.Neurite() & 'neurite_type="axon"').proj(presyn='segment_id') * ta3p100.CurrentSegmentation()
post_trace_segments = (ta3p100.Segment() & ta3p100.CurrentSegmentation() & ndap100.Trace()).proj(postsyn='segment_id')
post_trace_segments

segmentation segmentation id,postsyn segment id unique within each Segmentation
2,648518346341352006
2,648518346341352223
2,648518346341353019
2,648518346341353186
2,648518346341353521
2,648518346341353574
2,648518346341353607


In [9]:
# Compartment table
final_synapses_with_comp_labels = ((ta3p100.Synapse() & post_trace_segments & axon_segments) - ta3p100.SynapseExclude().proj()) * ta3p100.SynapseCompartmentLabel
relevant_synapses_with_comp_labels = final_synapses_with_comp_labels.proj("presyn","postsyn","postsynaptic_label")
complete_synapse_comp_table = ((relevant_synapses_with_comp_labels * ta3p100.LabelKey().proj('description', postsynaptic_label='numeric')).proj("presyn","postsynaptic_label","description"))
complete_synapse_comp_table

segmentation segmentation id,synapse_id synapse index within the segmentation,postsynaptic_label numeric label of the compartment,presyn segment id unique within each Segmentation,description descriptive name of the label
2,3151603,1,648518346342923546,label_removed
2,387206,1,648518346349374995,label_removed
2,1594527,2,648518346341412676,Apical
2,3373341,2,648518346349371036,Apical
2,1184862,2,648518346349381459,Apical
2,2013965,2,648518346342918671,Apical
2,912163,2,648518346349373229,Apical


In [10]:

# Spine table
final_synapses_with_spine_labels = ((ta3p100.Synapse() & post_trace_segments & axon_segments) - ta3p100.SynapseExclude().proj()) * ta3p100.SynapseSpineLabel
relevant_synapses_with_spine_labels = final_synapses_with_spine_labels.proj("presyn","postsyn","spine_label")
complete_synapse_spine_table = ((relevant_synapses_with_spine_labels * ta3p100.LabelKey().proj('description', spine_label='numeric')).proj("presyn","spine_label","description"))
complete_synapse_spine_table


segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment,presyn segment id unique within each Segmentation,description descriptive name of the label
2,1148008,13,648518346349373161,Spine Head
2,627674,13,648518346349377159,Spine Head
2,2338583,13,648518346346588572,Spine Head
2,1623829,13,648518346348105139,Spine Head
2,3344006,13,648518346349383797,Spine Head
2,1639105,13,648518346342793138,Spine Head
2,1524653,13,648518346342795260,Spine Head


In [16]:
# Combine Label and Spine table
entries_to_add = dj.U('spine_label')*((dj.U('segmentation', 'synapse_id') & complete_synapse_comp_table) - (dj.U('segmentation', 'synapse_id') & complete_synapse_spine_table)).proj(spine_label='0')
entries_to_add

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label calculated attribute
2,387206,0
2,3151603,0
2,3394955,0
2,1579870,0
2,2829945,0
2,2903185,0
2,3266287,0


In [17]:
updated_spine_table = complete_synapse_spine_table.proj() + entries_to_add.proj()
updated_spine_table

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment
2,2192161,13
2,395097,13
2,155633,13
2,3190061,13
2,729367,13
2,38959,13
2,2562216,13


In [18]:
complete_spine_compartment_label_table = (relevant_synapses_with_comp_labels.proj('presyn', 'postsynaptic_label') * updated_spine_table).proj('presyn', compartment_label='postsynaptic_label')
complete_spine_compartment_label_table

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment,presyn segment id unique within each Segmentation,compartment_label numeric label of the compartment
2,787061,13,648518346349384441,3
2,1919091,13,648518346349471238,4
2,1230023,13,648518346341406867,3
2,3265837,13,648518346341409014,3
2,598572,13,648518346346297684,3
2,920532,13,648518346346302628,3
2,2441593,13,648518346341399527,3


In [19]:
complete_spine_compartment_label_table.proj("compartment_label",segment_id="presyn")
complete_spine_compartment_label_table

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment,presyn segment id unique within each Segmentation,compartment_label numeric label of the compartment
2,787061,13,648518346349384441,3
2,1919091,13,648518346349471238,4
2,1230023,13,648518346341406867,3
2,3265837,13,648518346341409014,3
2,598572,13,648518346346297684,3
2,920532,13,648518346346302628,3
2,2441593,13,648518346341399527,3


In [ ]:
#default storage
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float))
    return np.array(bones)